# 第7章 文本数据

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.__version__

'1.0.5'

## 六、问题与练习
### 1. 问题

#### 【问题一】 str对象方法和df/Series对象方法有什么区别？
str对象类似字符串数组，注重分散化，操作时自动扩散（例如练习一）；Series更强调整体性，需结合.apply使用
#### 【问题二】 给出一列string类型，如何判断单元格是否是数值型数据？ 
`.str.contains(r"^(-?\d+)(\.\d*)?$")`
#### 【问题三】 rsplit方法的作用是什么？它在什么场合下适用？
rsplit倒序拆分（参数n为最大拆分个数，取默认值时与split效果相同）。适合处理尾端有固定格式的字符串。url、email等。
#### 【问题四】 在本章的第二到第四节分别介绍了字符串类型的5类操作，请思考它们各自应用于什么场景？
- 拆分与拼接：提取、组织具有固定格式的字符串，如电话、地址、邮箱等。
- 替换：同一种信息的不同表示（数字转中文）
- 子串匹配与提取：提取关键字
- 过滤：大小写、数字过滤，统一字符串书写规则（首字母大写）。

### 2. 练习
#### 【练习一】 现有一份关于字符串的数据集，请解决以下问题：（正则表达式不熟，本题使用了参考答案，跪谢作者）
#### （a）现对字符串编码存储人员信息（在编号后添加ID列），使用如下格式：“×××（名字）：×国人，性别×，生于×年×月×日”
#### （b）将（a）中的人员生日信息部分修改为用中文表示（如一九七四年十月二十三日），其余返回格式不变。
#### （c）将（b）中的ID列结果拆分为原列表相应的5列，并使用equals检验是否一致。

In [62]:
#（a）现对字符串编码存储人员信息（在编号后添加ID列），使用如下格式：“×××（名字）：×国人，性别×，生于×年×月×日”
# 考察点：string类型的Series可以直接按字符串操作
df = pd.read_csv('data/String_data_one.csv',index_col='人员编号').astype('str').astype('string')#转为string格式
df.head()

df_str1 = (df['姓名']+':'+df['国籍']+'国人，性别'
          +df['性别']+'，生于'
          +df['出生年']+'年'
          +df['出生月']+'月'+df['出生日']+'日').to_frame().rename(columns={0:'ID'})
#string拼接成Series，转df，更换列名为ID
df_str1.head()

,ID
人员编号,
1,aesfd:2国人，性别男，生于1942年8月10日
2,fasefa:5国人，性别女，生于1985年10月4日
3,aeagd:4国人，性别女，生于1946年10月15日
4,aef:4国人，性别男，生于1999年5月13日
5,eaf:1国人，性别女，生于2010年6月24日


In [63]:
#（b）将（a）中的人员生日信息部分修改为用中文表示（如一九七四年十月二十三日），其余返回格式不变。
# 考察点：replace的使用
L_year = list('零一二三四五六七八九')
L_one = [s.strip() for s in list('  二三四五六七八九')]#['', '', '二', '三', '四', '五', '六', '七', '八', '九']
L_two = [s.strip() for s in list(' 一二三四五六七八九')]#['', '一', '二', '三', '四', '五', '六', '七', '八', '九']
df_new = (df['姓名']+':'+df['国籍']+'国人，性别'+df['性别']+'，生于'
          +df['出生年'].str.replace(r'\d',lambda x:L_year[int(x.group(0))])+'年'
          +df['出生月'].apply(lambda x:x if len(x)==2 else '0'+x)\
                      .str.replace(r'(?P<one>[\d])(?P<two>\d?)',lambda x:L_one[int(x.group('one'))]
                      +bool(int(x.group('one')))*'十' +L_two[int(x.group('two'))])+'月'
          +df['出生日'].apply(lambda x:x if len(x)==2 else '0'+x)\
                      .str.replace(r'(?P<one>[\d])(?P<two>\d?)',lambda x:L_one[int(x.group('one'))]
                      +bool(int(x.group('one')))*'十' +L_two[int(x.group('two'))])+'日')\
          .to_frame().rename(columns={0:'ID'})#处理月份日期时，补足2位，统一处理
df_new.head()

,ID
人员编号,
1,aesfd:2国人，性别男，生于一九四二年八月十日
2,fasefa:5国人，性别女，生于一九八五年十月四日
3,aeagd:4国人，性别女，生于一九四六年十月十五日
4,aef:4国人，性别男，生于一九九九年五月十三日
5,eaf:1国人，性别女，生于二零一零年六月二十四日


In [64]:
#(c)将（b）中的ID列结果拆分为原列表相应的5列，并使用equals检验是否一致。
# 考察点：子串的匹配与提取
df_res = df_new['ID'].str.extract(r'(?P<姓名>[a-zA-Z]+):(?P<国籍>[\d])国人，性别(?P<性别>[\w])，生于(?P<出生年>[\w]{4})年(?P<出生月>[\w]+)月(?P<出生日>[\w]+)日')

dic_year = {i[0]:i[1] for i in zip(list('零一二三四五六七八九'),list('0123456789'))}
dic_two  = {i[0]:i[1] for i in zip(list('十一二三四五六七八九'),list('0123456789'))}
dic_one  = {'十':'1','二十':'2','三十':'3',None:''}
df_res['出生年'] = df_res['出生年'].str.replace(r'(\w)+',lambda x:''.join([dic_year[x.group(0)[i]] for i in range(4)]))
df_res['出生月'] = df_res['出生月'].str.replace(r'(?P<one>\w?十)?(?P<two>[\w])',lambda x:dic_one[x.group('one')]+dic_two[x.group('two')]).str.replace(r'^0','10')
df_res['出生日'] = df_res['出生日'].str.replace(r'(?P<one>\w?十)?(?P<two>[\w])',lambda x:dic_one[x.group('one')]+dic_two[x.group('two')]).str.replace(r'^0','10')
df_res.head()

,姓名,国籍,性别,出生年,出生月,出生日
人员编号,,,,,,
1,aesfd,2,男,1942,8,10
2,fasefa,5,女,1985,10,4
3,aeagd,4,女,1946,10,15
4,aef,4,男,1999,5,13
5,eaf,1,女,2010,6,24


In [65]:
df_res.equals(df)

True

#### 【练习二】 现有一份半虚拟的数据集，第一列包含了新型冠状病毒的一些新闻标题，请解决以下问题：
#### （a）选出所有关于北京市和上海市新闻标题的所在行。
#### （b）求col2的均值。
#### （c）求col3的均值。

In [66]:
df2 = pd.read_csv('data/String_data_two.csv')
df2.head()

,col1,col2,col3
0,鄂尔多斯市第2例确诊患者治愈出院,19,363.6923
1,云南新增2例，累计124例,-67,-152.281
2,武汉协和医院14名感染医护出院,-86,325.6221
3,山东新增9例，累计307例,-74,-204.9313
4,上海开学日期延至3月,-95,4.05


In [71]:
#(a)选出所有关于北京市和上海市新闻标题的所在行
df2_bjsh = df2[df2.col1.str.contains('上海')| df2.col1.str.contains('北京')]
df2_bjsh.head()

,col1,col2,col3
4,上海开学日期延至3月,-95,4.05
5,北京新增25例确诊病例，累计确诊253例,-4,-289.1719
6,上海新增10例，累计243例,2,-73.7105
36,上海新增14例累计233例,-55,-83
40,上海新增14例累计233例,-88,-99


In [72]:
#(2)求col2的均值
df2_bjsh.col2.astype(float).mean()

-20.152173913043477

In [73]:
#(3)求col3的均值
df2_bjsh['col3  '].astype(float).mean()

-6.998510869565218

In [70]:
#判断字符串是否数字
import re

def isnumber(num):
    regex = re.compile(r"^(-?\d+)(\.\d*)?$")
    if re.match(regex,num):
        return True
    else:
        return False
    
isnumber('-18.888a')

False